Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Search unstructured data with Vertex AI Search.

## Install python packages
* https://cloud.google.com/python/docs/reference/aiplatform/latest


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform \
                               google-cloud-discoveryengine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 20.1 MB/s eta 0:00:00


## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [ ]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik

Updated property [core/project].


## Set the environment on GCP Project


In [ ]:
MODEL_NAME="gemini-1.5-flash-002"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Tool
import vertexai.generative_models as generative_models

# Grounding service is still in preview.
from vertexai.preview.generative_models import grounding

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

### Vertex AI Search end point URL

In [ ]:
SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/engines/layout-parser_1718022685327/servingConfigs/default_search:search"


In [ ]:
import vertexai
import google
import google.oauth2.credentials
from google.auth import compute_engine
import google.auth.transport.requests
import requests
import json
import os

stream = os.popen('gcloud auth print-access-token')
credential_token = stream.read().strip()

# access_token = !gcloud auth application-default print-access-token
# credential_token = access_token[0]


## Search contexts from Vertex AI Search

### REST API Call to retrieve contexts from Vertex AI Search

In [ ]:
def retrieve_vertex_ai_search(question:str, search_url:str, page_size:int)->str:

  """ retrieve information from enterprise search ( discovery engine )"""

  # Create a credentials token to call a REST API
  headers = {
      "Authorization": "Bearer "+ credential_token,
      "Content-Type": "application/json"
  }

  query_dic ={
      "query": question,
      "page_size": str(page_size),
      "offset": 0,
      "contentSearchSpec":{  # https://cloud.google.com/generative-ai-app-builder/docs/reference/rest/v1/ContentSearchSpec
            "searchResultMode" : "CHUNKS",
            "chunkSpec" : {
                "numPreviousChunks" : 2,
                "numNextChunks" : 2
            }
      },
  }

  data = json.dumps(query_dic)

  # Encode data as UTF8
  data=data.encode("utf8")

  response = requests.post(search_url,headers=headers, data=data)

  print(response.text)
  return response.text

### Parsing a document come from Vertex AI Search

In [ ]:
def parse_chunks(response_text:str)->dict:
    import json
    """Parse response to build a conext to be sent to LLM"""

    dict_results = json.loads(response_text)

    index = 0
    search_results = {}

    if dict_results.get('results'):

        for result in dict_results['results']:

            item = {}

            chunk = result['chunk']
            item['title'] = chunk['documentMetadata']['title']
            item['uri'] = chunk['documentMetadata']['uri']
            item['content'] = chunk['content']

            # Chunks appear starting from those closest to the current Contents.
            try:
              if 'chunkMetadata' in chunk:
                if 'previousChunks' in chunk['chunkMetadata']:
                  for p_chunk in chunk['chunkMetadata']['previousChunks']:
                      item['content'] = p_chunk['content'] +"\n"+ item['content']

                if 'nextChunks' in chunk['chunkMetadata']:
                  for n_chunk in chunk['chunkMetadata']['nextChunks']:
                      item['content'] = item['content'] +"\n"+ n_chunk['content']
            except KeyError:
                pass

            search_results[f'results-{index}'] = item
            index = index+1

    return search_results

### Search context from Vertex AI Search

In [ ]:
question = "신용정보 보호법상 개인정보에 대한 정의를 말해주세요."

page_size = 2

searched_ctx = retrieve_vertex_ai_search(question, SEARCH_URL, page_size)
print(searched_ctx)

{
  "results": [
    {
      "chunk": {
        "name": "projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348/branches/0/documents/2/chunks/c1",
        "id": "c1",
        "content": "\n# 제1장 총칙\n신용정보의 이용 및 보호에 관한 법률 신용정보의 이용 및 보호에 관한 법률(약칭: 신용정보법) [시행 2023. 9. 15.] [법률 제19234호, 2023. 3. 14., 타법개정] 금융위원회 (금융데이터정책과) 02-2100-2624 제1조(목적) 이 법은 신용정보 관련 산업을 건전하게 육성하고 신용정보의 효율적 이용과 체계적 관리를 도모하며 신용 정보의 오용·남용으로부터 사생활의 비밀 등을 적절히 보호함으로써 건전한 신용질서를 확립하고 국민경제의 발전 에 이바지함을 목적으로 한다. \u003c개정 2020. 2. 4.\u003e 제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. \u003c개정 2011. 5. 19., 2015. 3. 11., 2020. 2. 4.\u003e 1. \"신용정보\"란 금융거래 등 상거래에서 거래 상대방의 신용을 판단할 때 필요한 정보로서 다음 각 목의 정보를 말 한다. 가. 특정 신용정보주체를 식별할 수 있는 정보(나목부터 마목까지의 어느 하나에 해당하는 정보와 결합되는 경우 만 신용정보에 해당한다) 나. 신용정보주체의 거래내용을 판단할 수 있는 정보 다. 신용정보주체의 신용도를 판단할 수 있는 정보 라. 신용정보주체의 신용거래능력을 판단할 수 있는 정보 마. 가목부터 라목까지의 정보 외에 신용정보주체의 신용을 판단할 때 필요한 정보 1의2. 제1호가목의 “특정 신용정보주체를 식별할 수 있는 정보\"란 다음 각 목의 정보를 말한다. 가. 살아 있는 개인에 관한 정보로서 다음

In [ ]:
context = parse_chunks(searched_ctx)

print(context)

{'results-0': {'title': '신용정보의 이용 및 보호에 관한 법률(법률)(제19234호)(20230915)', 'uri': 'gs://it_laws_kr/law_pdf/신용정보의 이용 및 보호에 관한 법률(법률)(제19234호)(20230915).pdf', 'content': '\n# 제1장 총칙\n신용정보의 이용 및 보호에 관한 법률 신용정보의 이용 및 보호에 관한 법률(약칭: 신용정보법) [시행 2023. 9. 15.] [법률 제19234호, 2023. 3. 14., 타법개정] 금융위원회 (금융데이터정책과) 02-2100-2624 제1조(목적) 이 법은 신용정보 관련 산업을 건전하게 육성하고 신용정보의 효율적 이용과 체계적 관리를 도모하며 신용 정보의 오용·남용으로부터 사생활의 비밀 등을 적절히 보호함으로써 건전한 신용질서를 확립하고 국민경제의 발전 에 이바지함을 목적으로 한다. <개정 2020. 2. 4.> 제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2011. 5. 19., 2015. 3. 11., 2020. 2. 4.> 1. "신용정보"란 금융거래 등 상거래에서 거래 상대방의 신용을 판단할 때 필요한 정보로서 다음 각 목의 정보를 말 한다. 가. 특정 신용정보주체를 식별할 수 있는 정보(나목부터 마목까지의 어느 하나에 해당하는 정보와 결합되는 경우 만 신용정보에 해당한다) 나. 신용정보주체의 거래내용을 판단할 수 있는 정보 다. 신용정보주체의 신용도를 판단할 수 있는 정보 라. 신용정보주체의 신용거래능력을 판단할 수 있는 정보 마. 가목부터 라목까지의 정보 외에 신용정보주체의 신용을 판단할 때 필요한 정보 1의2. 제1호가목의 “특정 신용정보주체를 식별할 수 있는 정보"란 다음 각 목의 정보를 말한다. 가. 살아 있는 개인에 관한 정보로서 다음 각각의 정보 1) 성명, 주소, 전화번호 및 그 밖에 이와 유사

## Reasoning result with LLM

### Function to call LLM

In [ ]:
def generate(query:str):
    """
    Generate a response from the model.

    query :
      query to be sent to the model

    Returns:
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

### Run example

In [ ]:
from time import perf_counter

t1_start = perf_counter()

prompt = f"""

  당신은 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {context}
  Question : {question}
  """

outcome = generate(prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))


Time : 3.5912273809999817 seconds




문서에 따르면, 신용정보보호법에서 '개인신용정보'는 기업 및 법인에 관한 정보를 제외한 살아있는 개인에 관한 신용정보로 정의됩니다.  구체적으로는 다음 두 가지 경우에 해당하는 정보를 말합니다.

1. 해당 정보의 성명, 주민등록번호 및 영상 등을 통하여 특정 개인을 알아볼 수 있는 정보.
2. 해당 정보만으로는 특정 개인을 알아볼 수 없더라도 다른 정보와 쉽게 결합하여 특정 개인을 알아볼 수 있는 정보.


요약하자면, 개인을 식별 가능하게 하는 정보 또는 다른 정보와 결합하여 식별 가능하게 하는 정보가 개인신용정보에 해당한다고 법률에서 정의하고 있습니다.
